## Retention Schedule Checker Program

The main objectives of the Retention program are to:

1. Pull in the sample doc data from the generated Sample Doc csv
2. Clean up the document strings so they can be filtered
3. Loop through each sample doc line and filter out any that meet certain trigger criteria based on document type codes
4. Loop through the docs identified as "eligible for destruction" to identify any with legal holds (selected at random) 
5. Print final list of eligible docs to a csv outfile

## The Program

Like with the Sample Doc Generator, I start with calling the appropriate modules and laying out the foundation code. 

In [ ]:
from datetime import datetime
from datetime import timedelta
import random
import csv
current_date = datetime.today()
print("today's date =", current_date)

infile = open('sample_docs.csv', 'r')
doc_list = infile.readlines()
infile.close()

outfile = open('eligible_for_destruction.csv', 'w')
doc_csv = csv.writer(outfile) #csv writer function

One more step before executing the main program is to clean up the data pulled in from the csv. 

In [ ]:
doc_splits = []
for doc in doc_list[1:]:
    clean_one = doc.replace("[",'')
    clean_two = clean_one.replace("]", '')
    clean_three = clean_two.replace ("'",'')
    clean_split = clean_three.split(",")
    doc_splits.append(clean_split)

This removes any extra brackets and quotations that were leftover in the line of data. I also split the strings into lists so each piece of data is its own string. 

Now I can run the main program!

In [ ]:
retention_accumulator = []

# ADM440: Doc Date + 3 Months
for a in doc_splits:
    doc_date = a[1]
    doc_date_good = datetime.strptime(doc_date," %Y-%m-%d %H:%M:%S")
    if a[0] == "ADM440":
        if (doc_date_good + timedelta(days=730)) < current_date:
            retention_accumulator.append(a)

I set up the main accumulator and a loop to run through my clean doc list. I need to convert the doc date into a datetime object in order to do any date-related computations so I do that by pulling out the [1] position of each doc. 

Next, I filter through each doc and look at the Doc Type (position[0]) and use the filter to check the retention trigger based on that. If the doc meets the criteria that makes it eligible for destruction, the list is added to the accumulator loop. 

Now we have to check for "legal holds." 

In [ ]:
status_ref = ("Y","N")
for doc in retention_accumulator: #filter through for "legal holds"
    eligible_no_holds = []
    legal_hold = (random.choice(status_ref)) #is the doc on legal hold?
    if legal_hold  == "N":
        eligible_no_holds.append(doc)
        print(eligible_no_holds, file = outfile)

Similar to the Sample Generator's random assignment of account open/closed, this loop assigns a Y or N to randomly select docs on our list for "legal hold." In reality this would be reflected by a separate list of specific docs but for these purposes they are selected at random. 

If "on hold" the for loop skips over the docs. If not on hold, the loop prints the doc data to our final outfile which serves as our ultimate list of docs currently eligible for destruction. 

[Next: GUI](https://github.com/slryb9008/rybicki1_final_project_IS452/blob/master/GUI.ipynb)